In [1]:
from concurrent.futures import ThreadPoolExecutor
import os
import re
import numpy as np
from skimage.filters import gaussian
import tifffile
from skimage import io, filters, exposure, morphology, util
from skimage.morphology import binary_closing, binary_opening, ball, remove_small_holes, remove_small_objects
from skimage.segmentation import clear_border
import matplotlib
from matplotlib.backends.backend_agg import FigureCanvasAgg
from matplotlib.figure import Figure



In [2]:
# creates a list of paths (path_list) with working files

import os

# Create a list of paths the name of which has two variables: mNRA_Name and SubSET
# def get_path(mNRA_Red, mRNA_Green, SubSET):

def get_path():


    input_dir = fr'\\?\G:\Nikon_Data_2018-2020\R35_Feb2024\PLASec61'
    path_list = []

    # Loop through all directories and subdirectories inside the input directory
    for root, dirs, files in os.walk(input_dir):
        # Loop through all files in the current directory
        for file in files:
            # Check if the file has an _info.txt extension and starts with the folder name
            if "_info.txt" in file and file.startswith(os.path.basename(root)):
                
                # Read the file content and convert it to lowercase
                
                path_list.append(root)

    # Return the list of paths
    return path_list


In [3]:
# runs function get_path and saves path list as .txt file

path_list = get_path()

with open('path_list.txt', 'w') as file:
    for item in path_list:
        file.write(item + '\n')
        
len(path_list)

6

In [5]:
# path_list # to check the nimber of folders

In [10]:
# function for parallelization that takes masked and raw images of mRNA from the path list and labels puncta

def normalize_image(file_path):
    image = tifffile.imread(file_path)

    # Convert to float for processing
    img_3d = image.astype(np.float32)  # Use float32 for memory efficiency

    # Normalize each image based on its own max value
    # max_val = img_3d.mean() * 5 # Cell Bodies
    max_val = img_3d.max() # mRNA and Protein 
    # max_val = img_3d.max() / 2 # Nuclei 
    
    if max_val > 0:  # Avoid division by zero
        img_3d /= max_val
        
    return img_3d


def Nikon_segment_cell_body(path):
    
    Body_dir = fr'\\?\G:\Nikon_Data_2018-2020\R35_Feb2024\PLASec61\Body'
    
    # Check if the directory exists, and create it if it does not
    if not os.path.exists(Body_dir):
        os.makedirs(Body_dir, exist_ok=True)

    try:
        with open(os.path.join(path, os.path.basename(path) + '_info.txt'), 'r', encoding='utf-8', errors='ignore') as file:
            content = file.read()

        # # if you have 3 channels:
        # # Extract channel numbers and names using regular expressions
        channel_numbers = list(map(int, re.findall(r'ChannelNumbers:(\d+),(\d+),(\d+)', content)[0]))
        channel_names = re.findall(r'ChannelNames:(\w+),(\w+),(\w+)', content)[0]

        # uncomment if you have 4 channels:
        # channel_numbers = list(map(int, re.findall(r'ChannelNumbers:(\d+),(\d+),(\d+),(\d+)', content)[0])) #ONLY for 3rd channel
        # channel_names = re.findall(r'ChannelNames:(\w+),(\w+),(\w+),(\w+)', content)[0] #ONLY for 3rd channel
        
        # uncomment if you have 1 channel:
        # channel_numbers = list(map(int, re.findall(r'ChannelNumbers:(\d+),(\d+)', content)[0]))
        # channel_names = re.findall(r'ChannelNames:(\w+),(\w+)', content)[0] 
                
        
        # Combine channel numbers and names into a dictionary
        channel_info = dict(zip(channel_numbers, channel_names))

        mask0 = channel_names[0]
        raw0 = channel_numbers[0]
        mask1 = channel_names[1]
        raw1 = channel_numbers[1]
        mask2 = channel_names[2]
        raw2 = channel_numbers[2]
#         mask3 = channel_names[3] # if you have 4 channels
#         raw3 = channel_numbers[3] # if you have 4 channels
        
        Nuclei_mask_path = os.path.join(path, os.path.basename(path) + f'_ch0{raw0}.tif')
        Body_mask_path = os.path.join(path, os.path.basename(path) + f'_ch0{raw0}.tif')
    
        # Load the raw 3D tif image
        # raw_image1 = tifffile.imread(Nuclei_mask_path)
        # raw_image4 = tifffile.imread(Body_mask_path)
        
        raw_image1 = normalize_image(Nuclei_mask_path)
        raw_image4 = normalize_image(Body_mask_path)

        # Apply a 3D Gaussian filter to the raw image (optional, but can help reduce noise)
        smoothed_image1 = gaussian(raw_image1, sigma=3)
        smoothed_image4 = gaussian(raw_image4, sigma=4)

        # Compute the Otsu's threshold value
        threshold_value1 = np.max(smoothed_image1) / 10 # parameters to change (for NIKON typically 4-6)
        threshold_value4 = np.mean(smoothed_image4) * 1.1 # parameters to change (for NIKON typically 0.5-2.5)

        # Create a binary mask by thresholding the (optionally smoothed) raw image
        binary_mask1 = smoothed_image1 > threshold_value1    
        binary_mask4 = smoothed_image4 > threshold_value4    

        # Perform the closing operation
        closed_binary_mask1 = binary_closing(binary_mask1, ball(4)) # "4" is typically works well for the most of the cell nuclei images
        draft_opened_binary_mask4 = binary_opening(binary_mask4, ball(2)) # "2" is typically works well for the most of the cell body images

        width_cell = 130
    
        binary_body_mask1 = remove_small_objects(draft_opened_binary_mask4, min_size=width_cell ** 3) # binary cytosol clean  
        closed_binary_body_mask = binary_closing(binary_body_mask1, ball(4)) # "4" is typically works well for the most of the cell body images
        binary_body_mask_cleaned = remove_small_holes(closed_binary_body_mask, area_threshold=width_cell ** 3) # binary body mask
        binary_body_mask = binary_opening(binary_body_mask_cleaned, ball(2)) # "2" is typically works well for the most of the cell body images

        # Create Body mask      
        binary_body_mask = binary_body_mask      
    
        # Create Nuclei mask
        width_nuclei = 30
        
        pre_binary_nuclei_mask_cleaned = remove_small_holes(closed_binary_mask1, area_threshold=width_nuclei ** 3) # binary body mask
        pre_binary_nuclei_mask = remove_small_objects(pre_binary_nuclei_mask_cleaned, min_size=width_nuclei ** 3) # binary cytosol clean        
        
        opened_binary_nuclei_mask = binary_opening(pre_binary_nuclei_mask, ball(4))        
        binary_nuclei_mask = opened_binary_nuclei_mask

        # Create Cytosol mask
        inverted_nuclei_mask = np.bitwise_not(binary_nuclei_mask) # Invert the nuclei mask
        binary_cytosol_mask = binary_body_mask * inverted_nuclei_mask
           
        # Create Nuclei MAX mask
        nuclei_mask_MAX = np.max(binary_nuclei_mask, axis=0)
        # Create Body MAX mask
        body_mask_MAX = np.max(binary_body_mask, axis=0)    
        # Create Cytosol MAX mask
        inverted_nuclei_mask_MAX = np.bitwise_not(nuclei_mask_MAX)
        cytosol_mask_MAX = inverted_nuclei_mask_MAX * body_mask_MAX
        
        directory_name = 'Segmented'
        segmented_dir_path = os.path.join(path, directory_name)

        # Add these lines
        if not os.path.exists(segmented_dir_path):
            os.makedirs(segmented_dir_path)

        #saves the segmented images in the same directory the processed images are
        Masked_image1_path = os.path.join(segmented_dir_path, f'Masked_Body_image.tif')
        Masked_image2_path = os.path.join(segmented_dir_path, f'Masked_Nuclei_image.tif')
        Masked_image3_path = os.path.join(segmented_dir_path, f'Masked_Cytosol_image.tif')
        
        io.imsave(Masked_image1_path, binary_body_mask)
        io.imsave(Masked_image2_path, binary_nuclei_mask)
        io.imsave(Masked_image3_path, binary_cytosol_mask)
        
        print(f"Masked image 4 saved in: {os.path.abspath(Masked_image2_path)}")
        
        # Set up the plot configurations
        plot_titles = [ f'Raw {mask0} {raw_image4.shape[1]}', f'Body {raw_image1.shape[1]}', f'Nuclei {raw_image1.shape[1]}', f'Cytosol {raw_image1.shape[1]}', f'draft_opened_binary_mask4 {raw_image1.shape[1]}', f'binary_body_mask {raw_image1.shape[1]}']
        images = [np.max(raw_image4, axis=0), body_mask_MAX, nuclei_mask_MAX, cytosol_mask_MAX, np.max(draft_opened_binary_mask4, axis=0), np.max(binary_body_mask, axis=0)]

        # Create a new figure and set the size
        fig = Figure(figsize=(10, 10))
        canvas = FigureCanvasAgg(fig)

        # Iterate through the images and add them as subplots
        for i, (image, title) in enumerate(zip(images, plot_titles)):
            ax = fig.add_subplot(2, 3, i+1)
            ax.imshow(image)
            ax.set_title(title)
            ax.axis('off')

        # Adjust the layout automatically
        fig.tight_layout()

        Body_path = os.path.join(Body_dir, os.path.basename(path) + '.png')

        # Save the plot as a PNG file with 300 DPI
        canvas.print_figure(Body_path, dpi=300)

        print(f"Masked image 5 saved in: {os.path.abspath(Body_path)}")
        
    except Exception as e:
        print(f"Error processing image at path {path}: {e}")
        
    return

In [11]:
# this code creates paths to the folders where the segmentation was bad

import os

def get_paths_from_png_files_with_bad_segmentation():
    input_dir = fr'\\?\G:\Nikon_Data_2018-2020\R35_Feb2024\PLASec61\Again'

    # Check if the directory exists, and create it if it does not
    if not os.path.exists(input_dir):
        os.makedirs(input_dir, exist_ok=True)
    
    path_list = []

    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if ".png" in file:
                # Remove the '.png' extension and the word 'Again' from the path
                modified_root = root.replace("Again", "")
                file_without_extension = os.path.splitext(file)[0]
                # Add the 'fr' prefix to the path when appending to path_list
                path_list.append(fr"{modified_root}{file_without_extension}")

    return path_list

# Get the path list with 'fr' prefix
path_list_with_bad_segmentation = get_paths_from_png_files_with_bad_segmentation()

# Print the path list to verify
for path in path_list_with_bad_segmentation:
    print(path)
print(len(path_list_with_bad_segmentation))

0


In [12]:
# function of parallelization of process_image() function

from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

def Nikon_segment_cell_body_PARALLEL(path_list):
    with ThreadPoolExecutor(max_workers=32) as executor: #specify the number of cpu cores you want to use
        futures = {executor.submit(Nikon_segment_cell_body, path): path for path in path_list}

        for future in tqdm(as_completed(futures), total=len(path_list), desc="Processing Images"):
            pass

In [13]:
len(path_list)

6

In [14]:
len(path_list_with_bad_segmentation)

0

In [15]:
Nikon_segment_cell_body_PARALLEL(path_list)

Processing Images:   0%|                                                                         | 0/6 [00:00<?, ?it/s]C:\Users\Exx\AppData\Local\Temp\ipykernel_18744\3891615872.py:127: UserWarning: \\?\G:\Nikon_Data_2018-2020\R35_Feb2024\PLASec61\2024-01-30_PLA RyR2Sec61a-488 and after a-actinin-568 fc_cell2 deconv\Segmented\Masked_Body_image.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  io.imsave(Masked_image1_path, binary_body_mask)
C:\Users\Exx\AppData\Local\Temp\ipykernel_18744\3891615872.py:128: UserWarning: \\?\G:\Nikon_Data_2018-2020\R35_Feb2024\PLASec61\2024-01-30_PLA RyR2Sec61a-488 and after a-actinin-568 fc_cell2 deconv\Segmented\Masked_Nuclei_image.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  io.imsave(Masked_image2_path, binary_nuclei_mask)
C:\Users\Exx\AppData\Local\Temp\ipykernel_18744\3891615872.py:1

Masked image 4 saved in: \\?\G:\Nikon_Data_2018-2020\R35_Feb2024\PLASec61\2024-01-30_PLA RyR2Sec61a-488 and after a-actinin-568 fc_cell2 deconv\Segmented\Masked_Nuclei_image.tif


Processing Images:  17%|██████████▊                                                      | 1/6 [00:25<02:08, 25.73s/it]

Masked image 5 saved in: \\?\G:\Nikon_Data_2018-2020\R35_Feb2024\PLASec61\Body\2024-01-30_PLA RyR2Sec61a-488 and after a-actinin-568 fc_cell2 deconv.png


C:\Users\Exx\AppData\Local\Temp\ipykernel_18744\3891615872.py:127: UserWarning: \\?\G:\Nikon_Data_2018-2020\R35_Feb2024\PLASec61\2024-01-1 WT F_PLA_SercaSec61-488_Actinin-568 C2_deconv\Segmented\Masked_Body_image.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  io.imsave(Masked_image1_path, binary_body_mask)
C:\Users\Exx\AppData\Local\Temp\ipykernel_18744\3891615872.py:128: UserWarning: \\?\G:\Nikon_Data_2018-2020\R35_Feb2024\PLASec61\2024-01-1 WT F_PLA_SercaSec61-488_Actinin-568 C2_deconv\Segmented\Masked_Nuclei_image.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  io.imsave(Masked_image2_path, binary_nuclei_mask)
C:\Users\Exx\AppData\Local\Temp\ipykernel_18744\3891615872.py:128: UserWarning: \\?\G:\Nikon_Data_2018-2020\R35_Feb2024\PLASec61\2024-01-1 WT F_PLA_SercaSec61-488_Actinin-568 C2_deconv\Segmented\Masked_Nuclei_i

...

Masked image 4 saved in: \\?\G:\Nikon_Data_2018-2020\R35_Feb2024\PLASec61\2024-01-30_PLA RyR2Sec61a-488 and after a-actinin-568 fc_cell1 deconv\Segmented\Masked_Nuclei_image.tif


C:\Users\Exx\AppData\Local\Temp\ipykernel_18744\3891615872.py:129: UserWarning: \\?\G:\Nikon_Data_2018-2020\R35_Feb2024\PLASec61\2024-01-1 WT F_PLA_SercaSec61-488_Actinin-568 C1_deconv\Segmented\Masked_Cytosol_image.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  io.imsave(Masked_image3_path, binary_cytosol_mask)
Processing Images:  83%|██████████████████████████████████████████████████████▏          | 5/6 [00:34<00:04,  4.34s/it]

Masked image 4 saved in: \\?\G:\Nikon_Data_2018-2020\R35_Feb2024\PLASec61\2024-01-1 WT F_PLA_SercaSec61-488_Actinin-568 C1_deconv\Segmented\Masked_Nuclei_image.tif
Masked image 5 saved in: \\?\G:\Nikon_Data_2018-2020\R35_Feb2024\PLASec61\Body\2024-01-30_PLA RyR2Sec61a-488 and after a-actinin-568 fc_cell1 deconv.png


Processing Images: 100%|█████████████████████████████████████████████████████████████████| 6/6 [00:35<00:00,  5.88s/it]

Masked image 5 saved in: \\?\G:\Nikon_Data_2018-2020\R35_Feb2024\PLASec61\Body\2024-01-1 WT F_PLA_SercaSec61-488_Actinin-568 C1_deconv.png


In [23]:
Nikon_segment_cell_body_PARALLEL(path_list_with_bad_segmentation)

Processing Images: 0it [00:00, ?it/s]
